In [1]:
import collections
import copy
import datetime
import itertools
import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
show_Ids = True
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay
Tournament = cibblbibbl.tournament.handler.default.Tournament
NormalPlayer = cibblbibbl.player.NormalPlayer
StarPlayer = cibblbibbl.player.StarPlayer

In [3]:
G.register_tournaments()

In [4]:
G.register_matchups()

In [5]:
G.cleanup_achievements()

In [5]:
G.register_achievements()

In [18]:
T = G.tournaments["47500"]

In [19]:
T.rawplayerperformances()

{NormalPlayer('12515846'): {'td': 2,
  'pass': 0,
  'comp': 0,
  'blocks': 24,
  'spp': 6,
  'rush': 25,
  'int': 0,
  'mvp': 0,
  'cas': 0,
  'fouls': 0,
  'turns': 111,
  'team': Team(922672),
  'name': 'Zunker',
  'type': 'R',
  'perf': [('D', 4098603),
   ('L', 4100099),
   ('L', 4103660),
   ('L', 4104628),
   ('W', 4105358),
   ('L', 4108204),
   ('W', 4110305)]},
 NormalPlayer('12515848'): {'td': 1,
  'pass': 0,
  'comp': 0,
  'blocks': 22,
  'spp': 3,
  'rush': 6,
  'int': 0,
  'mvp': 0,
  'cas': 0,
  'fouls': 0,
  'turns': 76,
  'team': Team(922672),
  'name': 'Grishen',
  'type': 'R',
  'perf': [('D', 4098603),
   ('L', 4100099),
   ('L', 4103660),
   ('L', 4104628),
   ('W', 4105358),
   ('L', 4108204)],
  'dead': [4108204, 1, 7, 'blocked', '12576997']},
 NormalPlayer('12515856'): {'td': 0,
  'pass': 3,
  'comp': 1,
  'blocks': 6,
  'spp': 1,
  'rush': 2,
  'int': 0,
  'mvp': 0,
  'cas': 0,
  'fouls': 0,
  'turns': 50,
  'team': Team(922672),
  'name': 'Goverk',
  'type': 'R

In [8]:
T.playerperformancesources()

{NormalPlayer('11754146'): {NormalPlayer('11754146')},
 NormalPlayer('11783522'): {NormalPlayer('11783522')},
 NormalPlayer('11967732'): {NormalPlayer('11967732')},
 NormalPlayer('12062178'): {NormalPlayer('12062178')},
 NormalPlayer('12223648'): {NormalPlayer('12223648')},
 NormalPlayer('12255081'): {NormalPlayer('12255081')},
 NormalPlayer('12496458'): {NormalPlayer('12496458')},
 NormalPlayer('12514876'): {NormalPlayer('12514876')},
 NormalPlayer('12514881'): {NormalPlayer('12514881')},
 NormalPlayer('12595023'): {NormalPlayer('12595023')},
 NormalPlayer('12615341'): {NormalPlayer('12615341')},
 NormalPlayer('12616471'): {NormalPlayer('12616471')},
 NormalPlayer('12676525'): {NormalPlayer('12676525')},
 NormalPlayer('12103149'): {NormalPlayer('12103149')},
 NormalPlayer('12103152'): {NormalPlayer('12103152')},
 NormalPlayer('12114193'): {NormalPlayer('12114193')},
 NormalPlayer('12143379'): {NormalPlayer('12143379')},
 NormalPlayer('12325545'): {NormalPlayer('12325545')},
 NormalPla

In [14]:
D = T.playerperformances()
L = list(D)
L

[NormalPlayer('11754146'),
 NormalPlayer('11783522'),
 NormalPlayer('11967732'),
 NormalPlayer('12062178'),
 NormalPlayer('12223648'),
 NormalPlayer('12255081'),
 NormalPlayer('12496458'),
 NormalPlayer('12514876'),
 NormalPlayer('12514881'),
 NormalPlayer('12595023'),
 NormalPlayer('12615341'),
 NormalPlayer('12616471'),
 NormalPlayer('12676525'),
 NormalPlayer('12103149'),
 NormalPlayer('12103152'),
 NormalPlayer('12114193'),
 NormalPlayer('12143379'),
 NormalPlayer('12325545'),
 NormalPlayer('12407510'),
 NormalPlayer('12465025'),
 NormalPlayer('12494315'),
 NormalPlayer('12503457'),
 NormalPlayer('12533831'),
 NormalPlayer('12608838'),
 NormalPlayer('12664730'),
 NormalPlayer('12696907'),
 NormalPlayer('12712713'),
 NormalPlayer('12730313'),
 NormalPlayer('12139099'),
 NormalPlayer('12165569'),
 NormalPlayer('12362498'),
 NormalPlayer('12488193'),
 NormalPlayer('12502657'),
 NormalPlayer('12535865'),
 NormalPlayer('12536828'),
 NormalPlayer('12578862'),
 NormalPlayer('12695465'),
 

In [15]:
L[-1]

RaisedDeadPlayer('RAISED-7154_12640386_0')

In [16]:
D[_]

{'cas': 1,
 'rush': 27,
 'int': 0,
 'mvp': 1,
 'td': 1,
 'turns': 89,
 'blocks': 25,
 'pass': 0,
 'comp': 0,
 'fouls': 0,
 'spp': 10,
 'perf': [('L', 4101812),
  ('W', 4103804),
  ('L', 4105589),
  ('L', 4108012),
  ('L', 4109992),
  ('L', 4112185),
  ('D', 4114822)],
 'team': Team(896228),
 'name': 'Sinbad',
 'type': 'D',
 'retired': True,
 'retiredlast': True}

In [7]:
def _playersseq(T, source_playersseq):
  StarPlayer = cibblbibbl.player.StarPlayer
  players = []
  for Pl in sorted(source_playersseq, key=lambda Pl: Pl.name):
    if Pl.achievements:
      prestige = sum(
          A.prestige(T.season) for A in Pl.achievements
      )
      if prestige or isinstance(Pl, StarPlayer):
        players.append([Pl, prestige])
    elif isinstance(Pl, StarPlayer):
      players.append([Pl, 0])
  return players

In [22]:
deadplayers = _playersseq(T, T.deadplayers())
deadplayers

[[NormalPlayer('12514748'), 10],
 [NormalPlayer('12612939'), 10],
 [NormalPlayer('12469185'), 40],
 [NormalPlayer('12514596'), 20]]

In [20]:
transferred = T.transferredplayers()
trplayers = _playersseq(T, transferred)
trplayers

[]

In [21]:
retiredplayers = T.retiredplayers()
retplayers = _playersseq(T, retiredplayers)
retplayers

[]

In [23]:
cibblbibbl.bbcode.player(_[0][0])

'[url=/p/player?player_id=12514748]Asavar the Parasite[/url]'

In [9]:
T.retiredplayers()

{NormalPlayer('12616471'): {'td': 0,
  'pass': 0,
  'comp': 0,
  'blocks': 5,
  'spp': 0,
  'rush': 0,
  'int': 0,
  'mvp': 0,
  'cas': 0,
  'fouls': 1,
  'turns': 16,
  'perf': [('L', 4098581), ('L', 4102159), ('L', 4103695)],
  'name': 'Marano sul Panaro VI',
  'team': Team(874302),
  'type': 'R',
  'retired': True,
  'tournament': Tournament('cibbl', '47502')},
 NormalPlayer('12503457'): {'td': 0,
  'pass': 0,
  'comp': 0,
  'blocks': 1,
  'spp': 0,
  'rush': 0,
  'int': 0,
  'mvp': 0,
  'cas': 0,
  'fouls': 0,
  'turns': 4,
  'perf': [('W', 4098581)],
  'name': 'Vudiaz',
  'team': Team(896228),
  'type': 'R',
  'retired': True,
  'tournament': Tournament('cibbl', '47502')},
 NormalPlayer('12362498'): {'td': 1,
  'pass': 0,
  'comp': 0,
  'blocks': 5,
  'spp': 3,
  'rush': 8,
  'int': 0,
  'mvp': 0,
  'cas': 0,
  'fouls': 0,
  'turns': 22,
  'perf': [('W', 4099395), ('L', 4103804), ('L', 4105421), ('D', 4106517)],
  'name': 'Wuwuhuwu',
  'team': Team(897353),
  'type': 'R',
  'retir

In [12]:
players = T.deadplayers()
_playersseq(T, players)

[]

In [13]:
players

{NormalPlayer('12143379'): [4102141, 1, 3, 'crowdPushed', '12667617'],
 NormalPlayer('12139099'): [4103804, 2, 1, 'blocked', '12636777'],
 NormalPlayer('12536828'): [4103804, 2, 3, 'gfiFail', None],
 NormalPlayer('12255962'): [4099395, 2, 3, 'blocked', '12139099'],
 NormalPlayer('12492986'): [4103695, 2, 0, 'eaten', '12411917'],
 NormalPlayer('12545355'): [4105540, 1, 3, 'blocked', '12712713'],
 NormalPlayer('12597730'): [4105540, 1, 8, 'blocked', '12114193'],
 NormalPlayer('12615447'): [4099395, 1, 3, 'eaten', '12411917'],
 NormalPlayer('12622181'): [4105540, 1, 2, 'fouled', '12533831'],
 NormalPlayer('12180822'): [4106584, 2, 6, 'blocked', '12259937'],
 NormalPlayer('12588115'): [4109431, 1, 8, 'blocked', '12722742'],
 NormalPlayer('12848989'): [4109431, 2, 1, 'blocked', '12165569'],
 NormalPlayer('11977896'): [4101812, 1, 6, 'blocked', '12259937'],
 NormalPlayer('12664729'): [4114822, 2, 1, 'fouled', '12465025']}

In [9]:
T.bestplayersname

'Bronze Excellence'

In [10]:
T.bestplayers()

{'Blocks': NormalPlayer('11512890'),
 'Casualties': NormalPlayer('11590769'),
 'Completions': NormalPlayer('11582135'),
 'Fouls': NormalPlayer('11512909'),
 'Interceptions': NormalPlayer('11693781'),
 'MVP': None,
 'Pass': NormalPlayer('11657983'),
 'Rush': NormalPlayer('11436778'),
 'SPP': NormalPlayer('11512894'),
 'Touchdowns': NormalPlayer('11582133'),
 'Turns': None,
 'Blocking Thrower': None,
 'Triple': None,
 'Blocking Scorer': None,
 'Allrounder': None,
 'Scoring Thrower': NormalPlayer('11582133')}

In [11]:
T.extraplayerperformances(join=True)

{NormalPlayer('11439074'): {'int': 0,
  'spp': 0,
  'td': 0,
  'mvp': 0,
  'turns': 37,
  'cas': 0,
  'rush': 0,
  'pass': 0,
  'fouls': 0,
  'blocks': 16,
  'comp': 0,
  'perf': [('L', 3879088), ('L', 3883528), ('L', 3884415)],
  'team': Team(854507),
  'type': 'R',
  'name': "M'zart, the Unbound Flame",
  'dead': [3884415, 2, 2, 'blocked', '11512896'],
  'Blocking Thrower': 0,
  'Blocking Scorer': 0,
  'Scoring Thrower': 0,
  'Triple': 0,
  'Allrounder': 0},
 NormalPlayer('11439075'): {'int': 0,
  'spp': 17,
  'td': 1,
  'mvp': 2,
  'turns': 93,
  'cas': 2,
  'rush': 10,
  'pass': 0,
  'fouls': 0,
  'blocks': 53,
  'comp': 0,
  'perf': [('L', 3879088),
   ('L', 3883528),
   ('L', 3884415),
   ('L', 3888613),
   ('L', 3889654),
   ('W', 3890233)],
  'team': Team(854507),
  'type': 'R',
  'name': 'Eltrith, the Silent',
  'Blocking Thrower': 0,
  'Blocking Scorer': 1,
  'Scoring Thrower': 0,
  'Triple': 0,
  'Allrounder': 0},
 NormalPlayer('11439076'): {'int': 0,
  'spp': 2,
  'td': 0,


In [12]:
L = sorted(T.achievements)
for i, a in enumerate(L):
    print(i, a)

0 Match Result Prestige([42426] CIBBL - Year One, Autumn - Bronze Division, [854502] Spartak Bolgasgrad)
1 Match Result Prestige([42426] CIBBL - Year One, Autumn - Bronze Division, [859122] Flying Drop Bears)
2 Match Result Prestige([42426] CIBBL - Year One, Autumn - Bronze Division, [863441] Tor Anroc Anglers)
3 Match Result Prestige([42426] CIBBL - Year One, Autumn - Bronze Division, [862033] Forsaken of the Pantheon)
4 Match Result Prestige([42426] CIBBL - Year One, Autumn - Bronze Division, [854500] Seven Samples of Decay)
5 Match Result Prestige([42426] CIBBL - Year One, Autumn - Bronze Division, [864291] Norscan Curse)
6 Match Result Prestige([42426] CIBBL - Year One, Autumn - Bronze Division, [854507] Coven of the Mirrored Hall)
7 Standings Prestige([42426] CIBBL - Year One, Autumn - Bronze Division, [854502] Spartak Bolgasgrad)
8 Standings Prestige([42426] CIBBL - Year One, Autumn - Bronze Division, [859122] Flying Drop Bears)
9 Standings Prestige([42426] CIBBL - Year One, Autu

In [13]:
A = L[18]

In [14]:
A.config

{'categs': ['Fouls'], 'prestige': 10, 'prev_subjectId': 'RAISED-7154_STAR-Max Spleenripper_11699071', 'prev_tournamentId': '42426', 'status': 'transferred transferred awarded'}

In [19]:
S = cibblbibbl.season.Season("cibbl", 1, 4)

In [20]:
print(S)

Season('cibbl', 1, 4)


In [21]:
A.prestige(S)

0

In [22]:
status = 'transferred transferred awarded'

In [24]:
status

'awarded'

In [26]:
A.prestige(A.season)

0

In [28]:
status = A.get("status", "proposed")
status

'transferred transferred awarded'

In [29]:
while status.startswith("transferred "):
      status = status[len("transferred "):]
status

'awarded'

In [32]:
    if A.tournament.awarded:
      statuses = {"awarded"}
    else:
      statuses = {"awarded", "proposed"}
statuses

{'awarded'}

In [12]:
Ma = Match(Pl.config["prevdeadmatchId"])
Mu = Ma.matchup
T = self = Mu.tournament
T

NameError: name 'Pl' is not defined

In [13]:
print(cibblbibbl.tournament.export.playerperformances.plaintext.export(T))

             Name                             Team                ★P   Td   Cs   It   Cp   Fo   BS   BT   ST   Tr   Ar
Count Ludovig                    Tower of the Necromancer         28    5    1         1         1    1    1    1     
Daniel Gil                       Ninja Warriors                   22    2    3                   2                    
Nisadei                          Trantio Dragon Hawks             20    5                                             
Urifdaemonkin                    Crimson Skull                    20        10                                        
Shiamoto The Undying             Secret Cult Of Shiamoto          19         7                                        
Falandar                         Tor Anroc Anglers                18    4    2         2    3    2    2    2    2     
Count Yaroslav                   Tower of the Necromancer         17    4              5                   4          
Basilio Mortalmente              Montetriste Nob

In [3]:
Pl = player('RAISED-7392_12022010_12496642')  # raise on last match

In [ ]:
performances = T.extraplayerperformances(join=True)

In [ ]:
performances

In [8]:
Sjs = list(performances.keys())

In [16]:
performances = cibblbibbl.tournament.performance.trans(performances)
performances

{NormalPlayer('12219671'): {'Completions': 0,
  'Casualties': 12,
  'Interceptions': 0,
  'SPP': 35,
  'Blocks': 77,
  'Pass': 0,
  'MVP': 1,
  'Rush': 19,
  'Turnms': 101,
  'Fouls': 0,
  'Touchdowns': 2,
  'perf': [('D', 4024740),
   ('D', 4026666),
   ('W', 4029149),
   ('W', 4031992),
   ('L', 4033076),
   ('W', 4034603),
   ('W', 4037448)],
  'team': Team(903654),
  'type': 'R',
  'name': 'Itekuyet',
  'Blocking Thrower': 0,
  'Blocking Scorer': 2,
  'Scoring Thrower': 0,
  'Triple': 0,
  'Allrounder': 0},
 NormalPlayer('12219673'): {'Completions': 8,
  'Casualties': 1,
  'Interceptions': 0,
  'SPP': 16,
  'Blocks': 6,
  'Pass': 18,
  'MVP': 0,
  'Rush': 116,
  'Turnms': 99,
  'Fouls': 0,
  'Touchdowns': 2,
  'perf': [('D', 4024740),
   ('D', 4026666),
   ('W', 4029149),
   ('W', 4031992),
   ('L', 4033076),
   ('W', 4034603),
   ('W', 4037448)],
  'team': Team(903654),
  'type': 'R',
  'name': 'Ahmose',
  'Blocking Thrower': 1,
  'Blocking Scorer': 1,
  'Scoring Thrower': 2,
  'T

In [10]:
order = default_order = (
    "SPP",
    "Touchdowns",
    "Casualties",
    "Interceptions",
    "Completions",
    "Fouls",
    "Blocking Scorer",
    "Blocking Thrower",
    "Scoring Thrower",
    "Triple",
    "Allrounder",
)

In [11]:
performancekeytrans = {
    "Blocks": "blocks",
    "Touchdowns": "td",
    "SPP": "spp",
    "Fouls": "fouls",
    "Interceptions": "int",
    "Completions": "comp",
    "Casualties": "cas",
    "MVP": "mvp",
    "Pass": "pass",
    "Rush": "rush",
    "Turnms": "turns",
}

In [17]:
performances[Sjs[0]][order[0]]

35

In [19]:
ordered = sorted(performances,
    key=lambda Sj: tuple(performances[Sj][name] for name in order),
    reverse=True,
)

In [5]:
#T = self = G.tournaments["44074"]

In [6]:
standings = T.standings()
pprint(standings)

[{'cas': 14,
  'casdiff': -4,
  'cto': -1,
  'hth': -1,
  'perf': [('D', 4024740),
           ('D', 4026367),
           ('W', 4030781),
           ('W', 4030969),
           ('W', 4032572),
           ('D', 4033652),
           ('W', 4041379)],
  'prestige': 150,
  'pts': 15,
  'score': 8,
  'scorediff': 5,
  'td': 8,
  'tddiff': 5,
  'team': Team(910344)},
 {'cas': 17,
  'casdiff': 4,
  'cto': -1,
  'hth': -1,
  'perf': [('D', 4024740),
           ('D', 4026666),
           ('W', 4029149),
           ('W', 4031992),
           ('L', 4033076),
           ('W', 4034603),
           ('W', 4037448)],
  'prestige': 140,
  'pts': 14,
  'score': 14,
  'scorediff': 8,
  'td': 14,
  'tddiff': 8,
  'team': Team(903654)},
 {'cas': 14,
  'casdiff': -4,
  'cto': -1,
  'hth': 1,
  'perf': [('W', 4026342),
           ('D', 4029013),
           ('L', 4030781),
           ('L', 4031992),
           ('L', 4033320),
           ('W', 4036761),
           ('W', 4037369)],
  'prestige': 100,
  'pts': 10,


In [7]:
d = {}
for ds in self.standings():
    Te = ds["team"]
    perf = ds["perf"]
    for r, matchId in reversed(perf):
        if matchId:
            d[Te] = cibblbibbl.match.Match(matchId)
            break
    else:
        d[Te] = None

In [8]:
d

{Team(910344): Match(4041379),
 Team(903654): Match(4037448),
 Team(899682): Match(4037369),
 Team(883051): Match(4041379),
 Team(900747): Match(4037448),
 Team(896972): Match(4034549),
 Team(891237): Match(4037369),
 Team(897353): Match(4034549)}

In [9]:
list(reversed(perf))

[('W', 4034549),
 ('L', 4034337),
 ('D', 4032047),
 ('L', 4030969),
 ('L', 4029149),
 ('L', 4027540),
 ('L', 4026342)]

In [10]:
rawperf = T.rawplayerperformances()
pprint(rawperf)

lastretired Tournament('cibbl', '45883') NormalPlayer('12173153')
lastretired Tournament('cibbl', '45883') NormalPlayer('12450013')
{NormalPlayer('11892405'): {'blocks': 26,
                            'cas': 1,
                            'comp': 0,
                            'fouls': 0,
                            'int': 0,
                            'mvp': 0,
                            'name': '"Featherweight" Gunnar',
                            'pass': 0,
                            'perf': [('L', 4024847),
                                     ('D', 4029013),
                                     ('W', 4029542),
                                     ('L', 4031344),
                                     ('W', 4033076),
                                     ('W', 4034337),
                                     ('L', 4041379)],
                            'prespp': 586,
                            'rush': 32,
                            'spp': 8,
                            'td': 2,
  

                            'pass': -1,
                            'perf': [('L', 4024847),
                                     ('D', 4029013),
                                     ('W', 4029542),
                                     ('L', 4031344),
                                     ('W', 4033076),
                                     ('W', 4034337),
                                     ('L', 4041379)],
                            'prespp': 14,
                            'rush': 13,
                            'spp': 0,
                            'td': 0,
                            'team': Team(883051),
                            'turns': 65,
                            'type': 'R'},
 NormalPlayer('12173110'): {'blocks': 24,
                            'cas': 4,
                            'comp': 0,
                            'fouls': 0,
                            'int': 0,
                            'mvp': 0,
                            'name': 'Stonemuncher the Ancient',

                            'team': Team(910344),
                            'turns': 10,
                            'type': 'R'},
 NormalPlayer('12323847'): {'blocks': 7,
                            'cas': 1,
                            'comp': 0,
                            'fouls': 0,
                            'int': 0,
                            'mvp': 0,
                            'name': 'Lucien',
                            'pass': 0,
                            'perf': [('D', 4024740),
                                     ('D', 4026367),
                                     ('W', 4030781),
                                     ('W', 4030969),
                                     ('W', 4032572),
                                     ('D', 4033652),
                                     ('W', 4041379)],
                            'prespp': 14,
                            'rush': 0,
                            'spp': 2,
                            'td': 0,
                    

                                     ('W', 4034603),
                                     ('W', 4037448)],
                            'prespp': 0,
                            'rush': 0,
                            'spp': 5,
                            'td': 0,
                            'team': Team(903654),
                            'turns': 72,
                            'type': 'R'},
 NormalPlayer('12501592'): {'blocks': 2,
                            'cas': 0,
                            'comp': 0,
                            'dead': [1, 4, 'fouled', '12121960'],
                            'fouls': 0,
                            'int': 0,
                            'mvp': 0,
                            'name': 'Hashar Sharphorn',
                            'pass': 0,
                            'perf': [('W', 4027540)],
                            'prespp': 0,
                            'rush': 0,
                            'spp': 0,
                            'td': 0,
 

                                     'cas': 0,
                                     'comp': 0,
                                     'fouls': 0,
                                     'int': 0,
                                     'mvp': 0,
                                     'name': 'Merc Air Elemental 3',
                                     'pass': 0,
                                     'perf': [('L', 4027540)],
                                     'prespp': 0,
                                     'rush': 0,
                                     'spp': 0,
                                     'td': 0,
                                     'team': Team(897353),
                                     'turns': 3,
                                     'type': 'M'},
 MercenaryPlayer('MERC-7444-2913'): {'blocks': 4,
                                     'cas': 0,
                                     'comp': 0,
                                     'fouls': 0,
                                     '

In [11]:
Pl = player('12173153')
Pl.team.name

'Black Monolith Worshippers'

In [12]:
Pl.name

'Ghras Paingiver'

In [13]:
pprint(rawperf[player('12022010')])

{'blocks': 5,
 'cas': 0,
 'comp': 0,
 'dead': [2, 6, 'blocked', '12323837'],
 'fouls': 0,
 'int': 0,
 'mvp': 0,
 'name': 'Dew Midhon',
 'pass': 0,
 'perf': [('D', 4025662), ('D', 4026367)],
 'prespp': 42,
 'rush': 0,
 'spp': 0,
 'td': 0,
 'team': Team(891237),
 'turns': 28,
 'type': 'R'}


In [14]:
PS = T.playerperformancesources()
pprint(PS)

lastretired Tournament('cibbl', '45883') NormalPlayer('12173153')
lastretired Tournament('cibbl', '45883') NormalPlayer('12450013')
{NormalPlayer('11892405'): {NormalPlayer('11892405')},
 NormalPlayer('11892424'): {NormalPlayer('11892424')},
 NormalPlayer('11892428'): {NormalPlayer('11892428')},
 NormalPlayer('11892429'): {NormalPlayer('11892429')},
 NormalPlayer('11947871'): {NormalPlayer('11947871')},
 NormalPlayer('11950136'): {NormalPlayer('11950136')},
 NormalPlayer('12022005'): {NormalPlayer('12022005')},
 NormalPlayer('12022006'): {NormalPlayer('12022006')},
 NormalPlayer('12022008'): {NormalPlayer('12022008')},
 NormalPlayer('12022022'): {NormalPlayer('12022022')},
 NormalPlayer('12104982'): {NormalPlayer('12104982')},
 NormalPlayer('12117786'): {NormalPlayer('12117786')},
 NormalPlayer('12117788'): {NormalPlayer('12117788')},
 NormalPlayer('12117790'): {NormalPlayer('12117790')},
 NormalPlayer('12117791'): {NormalPlayer('12117791')},
 NormalPlayer('12117792'): {NormalPlayer('1

In [29]:
perf = T.playerperformances()
pprint(perf)

lastretired Tournament('cibbl', '43565') NormalPlayer('12004119')
{NormalPlayer('11590768'): {'blocks': 38,
                            'cas': 1,
                            'comp': 0,
                            'fouls': 0,
                            'int': 0,
                            'mvp': 2,
                            'name': 'S. Sang',
                            'pass': -1,
                            'perf': [('L', 3928115),
                                     ('L', 3929535),
                                     ('L', 3931198),
                                     ('L', 3935706),
                                     ('L', 3937119),
                                     ('L', 3939738)],
                            'prespp': 191,
                            'rush': 11,
                            'spp': 12,
                            'td': 0,
                            'team': Team(864291),
                            'turns': 74,
                            'type': 'R'},
 

 NormalPlayer('11714360'): {'blocks': 6,
                            'cas': 0,
                            'comp': 0,
                            'fouls': 10,
                            'int': 0,
                            'mvp': 1,
                            'name': 'Eratosthenes',
                            'pass': 0,
                            'perf': [('L', 3925834),
                                     ('L', 3927498),
                                     ('W', 3928397),
                                     ('W', 3931160),
                                     ('W', 3935706),
                                     ('L', 3936723)],
                            'prespp': 37,
                            'rush': 0,
                            'spp': 5,
                            'td': 0,
                            'team': Team(870410),
                            'turns': 40,
                            'type': 'R'},
 NormalPlayer('11721709'): {'blocks': 18,
                        

                            'pass': 0,
                            'perf': [('D', 3924541),
                                     ('W', 3927498),
                                     ('W', 3931198),
                                     ('W', 3936136),
                                     ('D', 3937370),
                                     ('W', 3939205)],
                            'prespp': 0,
                            'rush': 16,
                            'spp': 0,
                            'td': 0,
                            'team': Team(872475),
                            'turns': 46,
                            'type': 'R'},
 NormalPlayer('11864164'): {'blocks': 40,
                            'cas': 2,
                            'comp': 0,
                            'fouls': 0,
                            'int': 0,
                            'mvp': 2,
                            'name': 'Argorzzz Zumbbzzee',
                            'pass': 0,
                     

In [6]:
T = G.tournaments["44074"]

In [5]:
T.bestplayers()

{'Touchdowns': NormalPlayer('12030024'),
 'SPP': NormalPlayer('12030024'),
 'Fouls': NormalPlayer('12052334'),
 'Interceptions': NormalPlayer('11956884'),
 'Completions': NormalPlayer('11940513'),
 'Casualties': None,
 'Allrounder': NormalPlayer('11956884'),
 'Blocking Thrower': None,
 'Triple': NormalPlayer('11939705'),
 'Scoring Thrower': NormalPlayer('11938039'),
 'Blocking Scorer': NormalPlayer('12031251')}

In [7]:
eperf = T.extraplayerperformances()
pprint(eperf)

{NormalPlayer('11936932'): {'Allrounder': 0,
                            'Blocking Scorer': 0,
                            'Blocking Thrower': 0,
                            'Scoring Thrower': 0,
                            'Triple': 0},
 NormalPlayer('11936933'): {'Allrounder': 0,
                            'Blocking Scorer': 1,
                            'Blocking Thrower': 1,
                            'Scoring Thrower': 2,
                            'Triple': 1},
 NormalPlayer('11936937'): {'Allrounder': 0,
                            'Blocking Scorer': 0,
                            'Blocking Thrower': 0,
                            'Scoring Thrower': 1,
                            'Triple': 0},
 NormalPlayer('11936939'): {'Allrounder': 0,
                            'Blocking Scorer': 0,
                            'Blocking Thrower': 0,
                            'Scoring Thrower': 0,
                            'Triple': 0},
 NormalPlayer('11936944'): {'Allrounder': 0,
   

In [8]:
max(d["Triple"] for d in eperf.values())

2

In [9]:
for Pl in sorted(eperf, key=lambda Pl: eperf[Pl]["Triple"], reverse=True):
    print(Pl)
    print(eperf[Pl])
    print()

NormalPlayer('11939705')
{'Blocking Thrower': 2, 'Blocking Scorer': 2, 'Scoring Thrower': 2, 'Triple': 2, 'Allrounder': 0}

NormalPlayer('11936933')
{'Blocking Thrower': 1, 'Blocking Scorer': 1, 'Scoring Thrower': 2, 'Triple': 1, 'Allrounder': 0}

NormalPlayer('11974844')
{'Blocking Thrower': 2, 'Blocking Scorer': 1, 'Scoring Thrower': 1, 'Triple': 1, 'Allrounder': 0}

NormalPlayer('11939696')
{'Blocking Thrower': 1, 'Blocking Scorer': 1, 'Scoring Thrower': 1, 'Triple': 1, 'Allrounder': 0}

NormalPlayer('11956884')
{'Blocking Thrower': 1, 'Blocking Scorer': 1, 'Scoring Thrower': 1, 'Triple': 1, 'Allrounder': 1}

NormalPlayer('11936932')
{'Blocking Thrower': 0, 'Blocking Scorer': 0, 'Scoring Thrower': 0, 'Triple': 0, 'Allrounder': 0}

NormalPlayer('11936937')
{'Blocking Thrower': 0, 'Blocking Scorer': 0, 'Scoring Thrower': 1, 'Triple': 0, 'Allrounder': 0}

NormalPlayer('11936939')
{'Blocking Thrower': 0, 'Blocking Scorer': 0, 'Scoring Thrower': 0, 'Triple': 0, 'Allrounder': 0}

NormalPl

In [34]:
pprint(perf[player('12496642')])

KeyError: NormalPlayer('12496642')

In [35]:
Pl = player('12496642')

In [36]:
Pl.team.name

"Vlad's Vampheroes"

In [22]:
Pl.prev

RaisedDeadPlayer('RAISED-7392_12022010_12496642')

In [15]:
players = set(rawperf)

In [16]:
deadplayers = {Pl for Pl, d in rawperf.items() if d.get("dead")}
deadplayers

{NormalPlayer('11950136'),
 NormalPlayer('12022010'),
 NormalPlayer('12117786'),
 NormalPlayer('12117795'),
 NormalPlayer('12158266'),
 NormalPlayer('12158267'),
 NormalPlayer('12173143'),
 NormalPlayer('12219903'),
 NormalPlayer('12339037'),
 NormalPlayer('12346893'),
 NormalPlayer('12354844'),
 NormalPlayer('12438916'),
 NormalPlayer('12501592'),
 NormalPlayer('12519811'),
 MercenaryPlayer('MERC-7444-0015'),
 MercenaryPlayer('MERC-7444-1067')}

In [17]:
for Pl in deadplayers:
    print(Pl, Pl.nexts)

NormalPlayer('11950136') set()
NormalPlayer('12339037') set()
NormalPlayer('12158266') set()
NormalPlayer('12346893') set()
NormalPlayer('12117795') set()
NormalPlayer('12438916') set()
NormalPlayer('12173143') set()
NormalPlayer('12158267') set()
MercenaryPlayer('MERC-7444-1067') set()
NormalPlayer('12354844') set()
NormalPlayer('12519811') set()
NormalPlayer('12219903') set()
MercenaryPlayer('MERC-7444-0015') set()
NormalPlayer('12501592') set()
NormalPlayer('12022010') {RaisedDeadPlayer('RAISED-7392_12022010_12496642')}
NormalPlayer('12117786') set()


In [18]:
perfsources = {Pl: {Pl} for Pl in players}
delete = set()
for Pl in sorted(deadplayers):
    if Pl in delete:
        continue
    if not Pl.nexts:
        continue
    for Pl1 in Pl.nexts:
      if isinstance(Pl1, cibblbibbl.player.RaisedDeadPlayer) and Pl1.nexts:
        for Pl2 in Pl1.nexts:
            perfsources[Pl2] = perfsources.get(Pl2, set()) | {Pl, Pl1}
            delete |= {Pl, Pl1}
      else:
        perfsources[Pl1] = perfsources.get(Pl1, set()) | {Pl,}
        delete |= {Pl}
perfsources = {Pl: S for Pl, S in perfsources.items() if Pl not in delete} 
perfsources

{NormalPlayer('12504647'): {NormalPlayer('12504647')},
 MercenaryPlayer('MERC-7444-1240'): {MercenaryPlayer('MERC-7444-1240')},
 MercenaryPlayer('MERC-7444-3727'): {MercenaryPlayer('MERC-7444-3727')},
 MercenaryPlayer('MERC-7444-1790'): {MercenaryPlayer('MERC-7444-1790')},
 NormalPlayer('12130906'): {NormalPlayer('12130906')},
 NormalPlayer('12357064'): {NormalPlayer('12357064')},
 MercenaryPlayer('MERC-7444-2820'): {MercenaryPlayer('MERC-7444-2820')},
 NormalPlayer('12522638'): {NormalPlayer('12522638')},
 NormalPlayer('12510951'): {NormalPlayer('12510951')},
 NormalPlayer('12399358'): {NormalPlayer('12399358')},
 NormalPlayer('12173155'): {NormalPlayer('12173155')},
 NormalPlayer('12219681'): {NormalPlayer('12219681')},
 NormalPlayer('12121959'): {NormalPlayer('12121959')},
 MercenaryPlayer('MERC-7444-3467'): {MercenaryPlayer('MERC-7444-3467')},
 MercenaryPlayer('MERC-7444-1807'): {MercenaryPlayer('MERC-7444-1807')},
 NormalPlayer('12438916'): {NormalPlayer('12438916')},
 NormalPlaye

In [19]:
sorted(perfsources)

[NormalPlayer('11892405'),
 NormalPlayer('11892424'),
 NormalPlayer('11892428'),
 NormalPlayer('11892429'),
 NormalPlayer('11947871'),
 NormalPlayer('11950136'),
 NormalPlayer('12022005'),
 NormalPlayer('12022006'),
 NormalPlayer('12022008'),
 NormalPlayer('12022022'),
 NormalPlayer('12104982'),
 NormalPlayer('12117786'),
 NormalPlayer('12117788'),
 NormalPlayer('12117790'),
 NormalPlayer('12117791'),
 NormalPlayer('12117792'),
 NormalPlayer('12117795'),
 NormalPlayer('12117802'),
 NormalPlayer('12121959'),
 NormalPlayer('12121960'),
 NormalPlayer('12121974'),
 NormalPlayer('12130906'),
 NormalPlayer('12139096'),
 NormalPlayer('12139099'),
 NormalPlayer('12158266'),
 NormalPlayer('12158267'),
 NormalPlayer('12158269'),
 NormalPlayer('12158272'),
 NormalPlayer('12158279'),
 NormalPlayer('12162547'),
 NormalPlayer('12165569'),
 NormalPlayer('12169214'),
 NormalPlayer('12169228'),
 NormalPlayer('12173110'),
 NormalPlayer('12173143'),
 NormalPlayer('12173145'),
 NormalPlayer('12173151'),
 

In [18]:
cibblbibbl._init_existing_players

frozenset({RaisedDeadPlayer('RAISED-7154_11456003_11460719'),
           NormalPlayer('11460719'),
           RaisedDeadPlayer('RAISED-7154_STAR-7564_11699071'),
           NormalPlayer('11699071'),
           RaisedDeadPlayer('RAISED-7240_11709016_0'),
           NormalPlayer('11744536'),
           NormalPlayer('11744537'),
           NormalPlayer('11747061'),
           RaisedDeadPlayer('RAISED-7154_11624587_11747108'),
           NormalPlayer('11747108'),
           NormalPlayer('11765469'),
           NormalPlayer('11791157'),
           NormalPlayer('11814852'),
           RaisedDeadPlayer('RAISED-7385_STAR-7542_11850036'),
           NormalPlayer('11850036'),
           RaisedDeadPlayer('RAISED-7240_11903019_0'),
           RaisedDeadPlayer('RAISED-7385_11795536_11933869'),
           NormalPlayer('11933869'),
           RaisedDeadPlayer('RAISED-7385_11446521_11936975'),
           NormalPlayer('11936975'),
           NormalPlayer('11940785'),
           RaisedDeadPlayer('RAISED